In [ ]:
import numpy as np
import sys
import pickle
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from IPython.display import display
from argparse import ArgumentParser
import src.modules.processor_pred as Processor
import src.modules.model as Model

parser = ArgumentParser(description="Specifying Input Parameters")
parser.add_argument("-te", "--testfile_path", help="Specify the full path of the file with TCR sequences")
parser.add_argument("-mf", "--modelfile_path", help="Specify the full path of the file with trained model")
parser.add_argument("-c", "--chain", default="ce", help="Specify the chain (s) to use (ce, cem). Default: ce")
parser.add_argument("-o", "--result_path", default=sys.stdout, help="Specify output file")

class Args:
    def __init__(self, testfile_path, modelfile_path, chain,result_path):
        self.testfile_path = testfile_path 
        self.modelfile_path = modelfile_path
        self.chain = chain
        self.result_path=result_path

def Original_model_prediction(testfile_path,modelfile_path,result_path,chain):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(testfile_path = testfile_path,modelfile_path=modelfile_path,chain = chain,result_path=result_path)
    modelfile_path = args.modelfile_path
    chain = args.chain
    test = pd.read_csv(args.testfile_path)
    test['CDR3B'] = test['CDR3B'].apply(lambda x: x[1:-1] if isinstance(x, str) else x)
    test['CDR3A'] = test['CDR3A'].apply(lambda x: x[1:-1] if isinstance(x, str) else x)
    test.drop(['MHC'], axis=1, inplace=True)
    new_column_names = {'CDR3B': 'CDR3b',
                     'CDR3A': 'CDR3a',
                    'HLA_seq': 'MHC',
                    'Epitope': 'epitope',
                   'Affinity':'binder'}
    test = test.rename(columns=new_column_names)
    assert chain in ["ce","cem"]
    if chain not in ["ce","cem"]:
        print("Invalid chain. You can select ce (cdr3b+epitope), cem (cdr3b+epitope+mhc)")
    if chain=='ce':
        pX_test, py_test = Processor.dataRepresentationBlosum62WithoutMHCb(test), test[["binder"]]

        model_rf = pickle.load(open(modelfile_path, 'rb'))
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf, test, pX_test, py_test, args.result_path)
        print('Done!')

    else:
        pX_test_mhc, py_test_mhc = Processor.dataRepresentationBlosum62WithMHCb(test), test[["binder"]]

        model_rf_mhc = pickle.load(open(modelfile_path, 'rb'))
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf_mhc, test, pX_test_mhc, py_test_mhc, args.result_path)
        print('Done!')   
    result = pd.read_csv (result_path+'result.csv')
    result = result.rename(columns={'binder': 'y_true', 'predict_proba': 'y_prob', 'binder_pred': 'y_pred'})
    result.to_csv(result_path+'probability.csv')

In [ ]:
testfile_path="../data/test_CDR3B_others.csv"
modelfile_path="../Original_model/epiTCR_BH.pickle"
result_path="../result_path/Original_model_prediction"
chain='cem'
Original_model_prediction(testfile_path,modelfile_path,result_path,chain)